In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as PCA
import sys
%matplotlib inline

pd.set_option('display.mpl_style', 'default')

dfPolluters = pd.read_csv('honeypot_data/content_polluters_tweets.txt', sep='\t', names=['UserID', 'TweetID', 'Tweet', 'CreatedAt'], parse_dates=['CreatedAt'])
polluters = pd.read_csv('honeypot_data/content_polluters.txt', sep='\t', names=['UserID', 'CreatedAt', 'CollectedAt', 'NumberOfFollowings', 'NumberOfFollowers', 'NumberOfTweets', 'LengthOfScreenName', 'LengthOfDescriptionInUserProfile'], parse_dates=['CreatedAt', 'CollectedAt'])
pol_dow = pd.read_csv('created_data/polDOWeek.csv', index_col=0)

polluters = polluters.set_index('UserID')

dfLegit = pd.read_csv('honeypot_data/legitimate_users_tweets.txt', sep='\t', names=['UserID', 'TweetID', 'Tweet', 'CreatedAt'], parse_dates=['CreatedAt'])
legit = pd.read_csv('honeypot_data/legitimate_users.txt', sep='\t', names=['UserID', 'CreatedAt', 'CollectedAt', 'NumberOfFollowings', 'NumberOfFollowers', 'NumberOfTweets', 'LengthOfScreenName', 'LengthOfDescriptionInUserProfile'], parse_dates=['CreatedAt', 'CollectedAt'])
leg_dow = pd.read_csv('created_data/legDOWeek.csv', index_col=0)

legit = legit.set_index('UserID')

/Users/reedmershon/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


Tweets with links freq
Tweets with unique links freq
Tweets with username freq
Tweets with unique username freq

Total Tweets

Use the sentiment analysis

Tweets perDay


Combine all of these into one single DataFrame



In [34]:
polluters = polluters.drop(['CreatedAt', 'CollectedAt', 'NumberOfFollowings', 'NumberOfFollowers', 'LengthOfScreenName', 'LengthOfDescriptionInUserProfile'], axis=1)
legit = legit.drop(['CreatedAt', 'CollectedAt', 'NumberOfFollowings', 'NumberOfFollowers', 'LengthOfScreenName', 'LengthOfDescriptionInUserProfile'], axis=1)

polluters = pd.concat([polluters, pol_dow], axis=1, join_axes=[polluters.index])
legit = pd.concat([legit, leg_dow], axis=1, join_axes=[legit.index])


The below code will drop all the accounts with less than 20 tweets collected. We did this already for the day of week parsing, but this will be appended to other accounts so we can just drop them now. 

In [35]:
polluters = polluters.dropna()
legit = legit.dropna()

# Collect data for Usernames and Links



In [36]:
import re

url = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
username = r'(?<=@)\w+'

def extractContent(users, tweets):
    
    usersFreq = {}
    
    for index, row in users.iterrows():
        
        totalTweets = row[0]
        urls = {}
        usernames = {}
        userTweetFreq = {'TotalLinks': 0, 'UniqueLinks': 0, 'TotalNames': 0, 'UniqueNames': 0}
        
        userData = tweets.loc[tweets['UserID'] == index]
        
        for tweet in userData['Tweet']:
            tweet = str(tweet)
            foundURLs = re.findall(url, tweet)
            foundNames = re.findall(username, tweet)

            for u in foundURLs:
                if u in urls:
                    urls[u] += 1
                else:
                    urls[u] = 1
                    
            for n in foundNames:
                if n in usernames:
                    usernames[n] += 1
                else:
                    usernames[n] = 1
                    
        totalUrls = sum(urls.values())
        totalNames = sum(usernames.values())
        uniqueUrls = len(urls)
        uniqueNames = len(usernames)
        
#         if totalNames >0:
#             print(usernames.keys())
#             print(usernames.values())
        
        freqUrl = totalUrls/totalTweets
        freqUniqueUrl = uniqueUrls/totalTweets
        freqName = totalNames/totalTweets
        freqUniqueName = uniqueNames/totalTweets
        
        userTweetFreq['TotalLinks'] = totalUrls
        userTweetFreq['FreqLinks'] = freqUrl
        userTweetFreq['UniqueLinks'] = freqUniqueUrl
        userTweetFreq['TotalNames'] = totalNames
        userTweetFreq['FreqNames'] = freqName
        userTweetFreq['UniqueNames'] = freqUniqueName
    
        usersFreq[index] = userTweetFreq
                    
    return pd.DataFrame.from_dict(usersFreq)

In [37]:
polFreq = extractContent(polluters, dfPolluters).T
legFreq = extractContent(legit, dfLegit).T

In [38]:
polluters = pd.concat([polluters, polFreq], axis=1, join_axes=[polluters.index])
legit = pd.concat([legit, legFreq], axis=1, join_axes=[legit.index])

In [39]:
polluters[:2]

,NumberOfTweets,Mon.,Tues.,Wed.,Thur.,Fri.,Sat.,Sun.,FreqLinks,FreqNames,TotalLinks,TotalNames,UniqueLinks,UniqueNames
UserID,,,,,,,,,,,,,,
6301,861,0.155000,0.330000,0.150000,0.145000,0.095000,0.030000,0.095000,0.163763,0.077816,141.0,67.0,0.148664,0.051103
10836,226,0.106061,0.171717,0.151515,0.146465,0.257576,0.055556,0.111111,0.575221,0.238938,130.0,54.0,0.446903,0.057522


In [40]:
polluters.to_csv('created_data/userContentPol.csv', encoding='utf-8')
legit.to_csv('created_data/userContentLeg.csv', encoding='utf-8')

In [43]:
print(len(legFreq))

18195


In [13]:
from sklearn.neighbors import KNeighborsClassifier as KNClass
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier #Neural Network

#needed for the plots
from sklearn.metrics import roc_curve, auc
import sys

#Redoing these functions to take in the fold number and then automate the list generation
#default fold number is 10

#take the average of the above list, then look for the best feature set
def getROC(df1, df2, fold=10):
    #Need to possibly take in a list then append the two data frames
    #then remove NaN
    
    if type(df1) is list:
        df1 = pd.concat(df1, axis=1, join_axes=[df1[0].index])
        df1 = df1.dropna(axis=0)
    if type(df2) is list:
        df2 = pd.concat(df2, axis=1, join_axes=[df2[0].index])
        df1 = df.dropna(axis=0)
    
    #section one, prepare data for the learning algorithms
    #divide into equal sections, take 10 total sets from each dataset
    #give proper classifications for each
    foldData = {}
    df1Segs = []
    df2Segs = []
    #add in a parameter for the number of folds to use
    segDF1 = int(len(df1)/fold)
    segDF2 = int(len(df2)/fold)
    
    for i in range(fold):
        foldData[i] = []
        df1Segs.append(i*segDF1)
        df2Segs.append(i*segDF2)

    for i in range(len(df1Segs)):
        #get the fold to fit on
        try:
            fitDF1 = df1[df1Segs[i]:df1Segs[i+1]]
            fitDF2 = df2[df2Segs[i]:df2Segs[i+1]]
            
            predDF1 = pd.concat([df1[:df1Segs[i]], df1[df1Segs[i+1]:]])
            predDF2 = pd.concat([df2[:df2Segs[i]], df2[df2Segs[i+1]:]])
        except IndexError: #Only for the last fold
            fitDF1 = df1[df1Segs[i]:]
            fitDF2 = df2[df2Segs[i]:]
            
            predDF1 = df1[:df1Segs[i]]
            predDF2 = df2[:df2Segs[i]]
            
        #use the DF to get the data
        fitData, fitCls = machLearnData(fitDF1, fitDF2)
        preData, preCls = machLearnData(predDF1, predDF2)
    
        #second one apply all algorithms to it
        #define the algorithms
        neigh = KNClass(n_neighbors=21)
        gnb = GaussianNB()
        dtc = DecisionTreeClassifier(random_state=0)
        vec = svm.SVC(probability=True)
        neur = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

        Algs = [neigh, gnb, dtc, vec, neur]#can add more later, also the order they are added to the dictionary
        
        for alg in Algs:
            alg.fit(fitData, fitCls)
            finCls = alg.predict_proba(preData)
            
            fpr, tpr, thresholds, roc_auc = probaTable(finCls)
            algList = [fpr, tpr, thresholds, roc_auc]
            
            foldData[i].append(algList)
        
    
    return foldData
    
    #get out the ROC curvs and AUC scores for each
    
    
#this will give the proper classifications
#dframe1 should be the positive df
def machLearnData(dframe1, dframe2, rng=0):
    data = []
    #0 = polluters
    #1 = legit
    classify = []
    for index, row in dframe1.iterrows():
        userList = []
        for i in range(len(row)):
            userList.append(row[i])
            
        data.append(userList)
        classify.append(0)
        
    for index, row in dframe2.iterrows():
        userList = []
        for i in range(len(row)):
            userList.append(row[i])

        data.append(userList)
        classify.append(1)
        
    return data, classify

def probaTable(fCls, posL=0, posCut=0.5):
    clsArray = []
    actualCls = []
    
    pos = []
    
    count = 0
    for line in fCls:
        if line[0] >= posCut:
            clsArray.append(0)
        else:
            clsArray.append(1)
        pos.append(line[0])
            
        if count < len(fCls)/2:
            actualCls.append(0)
            count += 1
        else:
            actualCls.append(1)
            
    #print(len(actualCls), len(fCls))
            
    #classTuple = zip(actualCls, fCls)

    fpr, tpr, thresholds = roc_curve(actualCls, pos, pos_label=posL)
    
    roc_auc = auc(fpr, tpr)
    
    #return {'polluter': [truePos, falseNeg], 'legit': [falsePos, trueNeg]}, acc, precision, recall, f1, fpr
    
    return fpr, tpr, thresholds, roc_auc




In [14]:
rocUC = getROC(polluters, legit, 10)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').